# Processamento de Linguagem Natural aplicada à Gestão Pública

Aula 5 (20/06): Classificação, Sumarização, Reconhecimento de Entidades Nomeadas

In [1]:
# Download dos dados
%%capture
!git clone https://github.com/samuelbarbosaa/oficina_nlp.git

In [2]:
%%capture
pip install transformers[sentencepiece]

In [3]:
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

plt.rcParams['figure.figsize'] = [10, 5]

In [4]:
import zipfile

with zipfile.ZipFile("/content/oficina_nlp/data/acordaos_cpf.zip", "r") as zip_ref:
    zip_ref.extractall("/content/oficina_nlp/data/")

In [5]:
acordaos = pd.read_csv("/content/oficina_nlp/data/acordaos_cpf.csv")[["acordao", "sumario"]].dropna()
acordaos

,acordao,sumario
0,"VISTOS, relatados e discutidos estes autos de ...",Tomada de Contas Especial. Subvenção social co...
1,"VISTOS, relatados e discutidos estes autos de ...",Tomada de Contas Especial. Subvenção social co...
2,"VISTOS, relatados e discutidos estes autos de ...",Relatório de Auditoria transformado em tomada ...
3,"VISTOS, relatados e discutidos estes autos de ...",Tomada de Contas Especial instaurada em cumpri...
4,"VISTOS, relatados e discutidos estes autos de ...",Tomada de Contas Especial instaurada em razão ...
...,...,...
29895,"VISTOS, relatados e discutidos estes autos de ...",TOMADA DE CONTAS ESPECIAL. CONVÊNIO PARA IMPLA...
29899,"VISTOS, relatados e discutidos estes autos de ...",TOMADA DE CONTAS ESPECIAL. RECEBIMENTO IRREGUL...
29900,"VISTOS, relatados e discutidos estes autos que...",REPRESENTAÇÃO. PAGAMENTO DE FUNÇÕES GRATIFICAD...
29904,"VISTOS, relatados e discutidos estes autos de ...",TOMADA DE CONTAS ESPECIAL. CONVERSÃO DE MONITO...


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [7]:
from transformers import pipeline

sumarizador = pipeline(task="summarization", model=model, tokenizer=tokenizer)
sumarizador(acordaos.iloc[500]["acordao"])

[{'summary_text': '<extra_id_0>, e o Tribunal de:'}]

In [8]:
acordaos.iloc[500]["acordao"]

'VISTOS, relatados e discutidos estes autos de concessão de aposentadorias, ACORDAM os Ministros do Tribunal de Contas da União, reunidos em Sessão da Primeira Câmara, diante das razões expostas pelo Relator, em:9.1. com fundamento nos incisos III e IX do art. 71 da Constituição Federal, c/c os arts. 1º, V, 39, II, e 45 da Lei nº 8.443/1992:9.1.1. considerar ilegais as concessões de aposentadoria em favor de Deolinda da Rocha Gonçalves (CPF 741.401.107-04) e Paulo José Viana Voto (CPF 741.849.847-04), e negar o registro dos respectivos atos;9.1.2. dispensar o ressarcimento das quantias indevidamente recebidas de boa-fé (Súmula 106 do TCU);9.1.3. determinar à unidade jurisdicionada que, no prazo de 15 (quinze) dias:9.1.3.1. dê ciência do inteiro teor desta deliberação, bem como do relatório e voto que a fundamentam, aos interessados cujos atos foram considerados ilegais;9.1.3.2. faça cessar os pagamentos decorrentes dos atos considerados ilegais, sob pena de responsabilidade solidária d

In [8]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
def prepara_base(textos, sumarios):
  dados = zip(textos, sumarios)
  return [{"summary": s, "text": t} for (t, s) in dados]

In [10]:
base = prepara_base(acordaos["acordao"], acordaos["sumario"])

In [11]:
len(base)

100

In [12]:
base[0]

{'summary': 'Tomada de Contas Especial. Subvenção social concedida pelo extinto  Ministério da Ação Social à SESNI. Irregularidades na aplicação dos  recursos transferidos. Citação. Revelia. Contas julgadas irregulares.  Débito. Autorização para a cobrança judicial da dívida. Remessa da  documentação pertinente ao Ministério Público da União. Determinação ao  Controle Interno. Encaminhamento de cópia à 5ªVara Federal de São João  de Meriti e à Procuradoria-Seccional da União em Petrópolis/RJ.',
 'text': 'VISTOS, relatados e discutidos estes autos de Tomada de Contas Especial, de responsabilidade do Sr. Fábio Gonçalves Raunheitti, então Presidente da Sociedade de Ensino Superior de Nova Iguaçu/SESNI/RJ, instaurada pela Delegacia Federal de Controle no Distrito Federal, em cumprimento à Decisão Plenária ¿ TCU nº 379/94, devido a constatação de irregularidades na aplicação dos recursos recebidos pela referida Sociedade do extinto Ministério da Ação Social, por meio da Subvenção Social nº 

In [13]:
import json

In [14]:
with open("/content/oficina_nlp/base.jsonl", "w") as fp:
  for x in base:
    json.dump(x, fp)
    fp.write('\n')

In [15]:
%%capture
!pip install datasets

In [16]:
from datasets import load_dataset

base_t = load_dataset("json", data_files="/content/oficina_nlp/base.jsonl", split="train")

Using custom data configuration default-5d34a674b806e5e8


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-5d34a674b806e5e8/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


In [17]:
base_t = base_t.train_test_split(test_size=0.2)

In [18]:
base_tokenizada = base_t.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7f95883720e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/oficina_nlp/resultado",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    #fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=base_tokenizada["train"],
    eval_dataset=base_tokenizada["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 80
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5
The following columns in the training set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: summary, text. If summary, text are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
